# MIDS 251 Final Project
## Analyzing Reddit Data
### Creating a prediction model for subreddit using pyspark.ml.DecisionTreeClassifier

In [1]:
import os
import sys

In [ ]:
! which python

In [ ]:
os.chdir('/usr/local/spark/')

In [ ]:
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.8.2.1-src.zip"))
os.chdir(spark_home)

In [5]:
execfile(os.path.join(spark_home, "python/pyspark/shell.py"))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.2
      /_/

Using Python version 2.7.5 (default, Jun 24 2015 00:41:19)
SparkContext available as sc, SQLContext available as sqlContext.


In [7]:
import os
os.getcwd()


'/usr/local/spark-1.5.3-SNAPSHOT-bin-spark-1.5.2-hadoop-2.6.2'

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.util import MLUtils

# Load and parse the data file, converting it to a DataFrame.
data = MLUtils.loadLibSVMFile(sc, "file:///usr/local/spark/data/mllib/sample_libsvm_data.txt").toDF()

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="precision")
accuracy = evaluator.evaluate(predictions)
print "Test Error = %g" % (1.0 - accuracy)

treeModel = model.stages[2]
print treeModel # summary only

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(692,[151,152,153...|
|       0.0|         1.0|(692,[154,155,156...|
|       1.0|         1.0|(692,[153,154,155...|
|       1.0|         1.0|(692,[126,127,128...|
|       0.0|         0.0|(692,[123,124,125...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0.037037
DecisionTreeClassificationModel of depth 2 with 5 nodes


In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *

sqlContext = SQLContext(sc)
     
data = sqlContext.read.
("swift://notebooks.spark/tweetsFull.parquet")
     
parquetFile.registerTempTable("tweets");
sqlContext.cacheTable("tweets")
tweets = sqlContext.sql("SELECT * FROM tweets")
print tweets.count()
tweets.cache()

2010_counts.tgz  2014_counts.tgz  set_java.sh	      zeromq-4.0.5.tar.gz
2011_counts.tgz  2015_counts.tgz  singleContainer.sh
2012_counts.tgz  master.txt	  test.py
2013_counts.tgz  reddit.files	  zeromq-4.0.5


In [8]:
quit()